# **Instalando e importando a biblioteca Pandas**

In [12]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd

# **Extraindo os dados**

In [27]:
arquivos = {
    '2021': 'dados/MICRODADOS_ENEM_2021.csv',
    '2022': 'dados/MICRODADOS_ENEM_2022.csv',
    '2023': 'dados/MICRODADOS_ENEM_2023.csv'
}

lista_dataframes = []

for ano, arquivo in arquivos.items():
    try:
        df = pd.read_csv(arquivo, encoding='latin-1', sep=';')
        lista_dataframes.append(df)
    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")

df = pd.concat(lista_dataframes, ignore_index=True)

# **Visualização geral**

In [15]:
display(df.head(5))
display(df.info())

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210053865474,2021,5,F,1,1,1,1,3,1,...,A,A,A,B,A,A,B,A,B,B
1,210052384164,2021,12,M,1,1,1,1,11,1,...,A,A,A,B,A,A,C,A,A,A
2,210052589243,2021,13,F,3,1,1,1,15,1,...,B,A,A,B,A,A,C,B,B,B
3,210052128335,2021,3,M,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,B,B
4,210051353021,2021,2,F,1,3,1,2,0,2,...,B,A,A,B,A,B,E,A,B,B


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6865937 entries, 0 to 6865936
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_INSCRICAO            int64  
 1   NU_ANO                  int64  
 2   TP_FAIXA_ETARIA         int64  
 3   TP_SEXO                 object 
 4   TP_ESTADO_CIVIL         int64  
 5   TP_COR_RACA             int64  
 6   TP_NACIONALIDADE        int64  
 7   TP_ST_CONCLUSAO         int64  
 8   TP_ANO_CONCLUIU         int64  
 9   TP_ESCOLA               int64  
 10  TP_ENSINO               float64
 11  IN_TREINEIRO            int64  
 12  CO_MUNICIPIO_ESC        float64
 13  NO_MUNICIPIO_ESC        object 
 14  CO_UF_ESC               float64
 15  SG_UF_ESC               object 
 16  TP_DEPENDENCIA_ADM_ESC  float64
 17  TP_LOCALIZACAO_ESC      float64
 18  TP_SIT_FUNC_ESC         float64
 19  CO_MUNICIPIO_PROVA      int64  
 20  NO_MUNICIPIO_PROVA      object 
 21  CO_UF_PROVA             int64  

None

## **Verificando valores nulos nas colunas**

In [31]:
print("Valores nulos por coluna:")
print(df.isnull().sum())

Valores nulos por coluna:


MemoryError: Unable to allocate 30.9 MiB for an array with shape (3, 10799892) and data type bool

In [ ]:
## **Verificando valores únicos nas colunas**

In [26]:
print("Valores únicos nas colunas:")
df.nunique(dropna=False)

Valores únicos nas colunas:


NU_INSCRICAO       6865937
NU_ANO                   2
TP_FAIXA_ETARIA         20
TP_SEXO                  2
TP_ESTADO_CIVIL          5
                    ...   
Q021                     3
Q022                     6
Q023                     3
qtd_computador           6
Q025                     3
Length: 76, dtype: int64

## **Transformação da coluna TB_ENSINO**

In [17]:
print("Valores únicos da coluna 'Ensino': \n", df['TP_ENSINO'].unique())
print("Coluna TP_ENSINO")
print(df['TP_ENSINO'].head(5))

Valores únicos da coluna 'Ensino': 
 [nan  1.  2.]
Coluna TP_ENSINO
0    NaN
1    NaN
2    NaN
3    1.0
4    1.0
Name: TP_ENSINO, dtype: float64


A coluna TB_ENSINO possui 3 valores, NaN, 1 e 2. Os valores NaN serão alterados para 0, para assim ser possível transformar o tipo para inteiro. Depois disso os valores serão alterados segundo a legenda dos microdados: 1 para Ensino Regulas e 2 para Educação Especial - Modalidade Substitutiva, o 0 será tratado como Não informado. O nome da tabela passará a ser ensino.

In [18]:
dicio_ensino = {
    0: 'Não informado',
    1: 'Ensino Regular',
    2: 'Educação Especial - Modalidade Substitutiva'
}

df['TP_ENSINO'] = (df['TP_ENSINO'].fillna(0).astype(int))

df['TP_ENSINO'] = df['TP_ENSINO'].map(dicio_ensino)

print("Coluna TP_ENSINO")
print(df['TP_ENSINO'].head(5))
print("Valores únicos da coluna 'TB_ENSINO': \n", df['TP_ENSINO'].unique())


Coluna TP_ENSINO
0     Não informado
1     Não informado
2     Não informado
3    Ensino Regular
4    Ensino Regular
Name: TP_ENSINO, dtype: object
Valores únicos da coluna 'TB_ENSINO': 
 ['Não informado' 'Ensino Regular'
 'Educação Especial - Modalidade Substitutiva']


In [19]:
df.rename(columns={'TP_ENSINO': 'ensino'}, inplace=True)
print("Coluna ensino")
print(df['ensino'].head(5))

Coluna ensino
0     Não informado
1     Não informado
2     Não informado
3    Ensino Regular
4    Ensino Regular
Name: ensino, dtype: object


## **Transformação da coluna Q006**

In [20]:
print("Valores únicos da coluna 'Q006': \n", df['Q006'].unique())
print('Coluna Q006')
print(df['Q006'].head(5))

Valores únicos da coluna 'Q006': 
 ['D' 'B' 'C' 'G' 'H' 'F' 'E' 'A' 'I' 'L' 'J' 'N' 'O' 'K' 'M' 'Q' 'P' nan]
Coluna Q006
0    D
1    B
2    C
3    B
4    C
Name: Q006, dtype: object


A coluna Q006 possui muitas respostas possíveis, indo do A ao Q, além do NaN. Por isso, agrupamos algumas das respostas possíveis, ficando: 'Nenhuma renda', 'Até R$ 1.320,00', 'De R$ 1.320,01 até R$ 3.400,00', 'De R$ 3.400,01 até R$ 8.100,00', 'De R$ 8.100,01 até R$ 25.200,00', 'Acima de R$ 25.200,01', os valores NaN serão tratados com 'Não informado'. As categorias foram alocadas de acordo com o valor mais próximo da resposta original. Além disso, trocamos o nome da coluna para 'renda_mensal'

In [21]:


df['Q006'] = df['Q006'].fillna('Não informado')

df.loc[df['Q006'] == 'A', 'Q006'] = 'Nenhuma renda'
df.loc[df['Q006'] == 'B', 'Q006'] = 'Até R$ 1.320,00'
df.loc[df['Q006'].isin(['C', 'D', 'E', 'F']), 'Q006'] = 'De R$ 1.320,01 até R$ 3.400,00'
df.loc[df['Q006'].isin(['G', 'H', 'I', 'J']), 'Q006'] = 'De R$ 3.400,01 até R$ 8.100,00'
df.loc[df['Q006'].isin(['K', 'L', 'M', 'N', 'O', 'P']), 'Q006'] = 'De R$ 8.100,01 até R$ 25.200,00'
df.loc[df['Q006'] == 'Q', 'Q006'] = 'Acima de R$ 25.200,01'


print("Coluna Q006")
print(df['Q006'].head(5))
print("Valores únicos da coluna 'Q006': \n", df['Q006'].unique())

Coluna Q006
0    De R$ 1.320,01 até R$ 3.400,00
1                   Até R$ 1.320,00
2    De R$ 1.320,01 até R$ 3.400,00
3                   Até R$ 1.320,00
4    De R$ 1.320,01 até R$ 3.400,00
Name: Q006, dtype: object
Valores únicos da coluna 'Q006': 
 ['De R$ 1.320,01 até R$ 3.400,00' 'Até R$ 1.320,00'
 'De R$ 3.400,01 até R$ 8.100,00' 'Nenhuma renda'
 'De R$ 8.100,01 até R$ 25.200,00' 'Acima de R$ 25.200,01' 'Não informado']


In [22]:
df.rename(columns={'Q006': 'renda_mensal'}, inplace=True)
print("Coluna renda_mensal")
print(df['renda_mensal'].head(5))

Coluna renda_mensal
0    De R$ 1.320,01 até R$ 3.400,00
1                   Até R$ 1.320,00
2    De R$ 1.320,01 até R$ 3.400,00
3                   Até R$ 1.320,00
4    De R$ 1.320,01 até R$ 3.400,00
Name: renda_mensal, dtype: object


## **Transformação da coluna Q024**

In [23]:
print("Valores únicos da coluna 'Q024': \n", df['Q024'].unique())
print('Coluna Q024')
print(df['Q024'].head(5))

Valores únicos da coluna 'Q024': 
 ['B' 'A' 'C' 'D' 'E' nan]
Coluna Q024
0    B
1    A
2    B
3    B
4    B
Name: Q024, dtype: object


A coluna Q024 possui 6 valores possíveis, as colunas com NaN foram preenchidas com Não informado, e cada uma das outras respostas foram trocadas pelas informações correspondes no dicionário de microdados, sendo elas: 'A' igual a 'Não.', 'B' igual a 'Sim, um.', 'C' igual a 'Sim, dois.', 'D' igual a 'Sim, três.', 'E' igual a 'Sim, quatro ou mais.', 'F' igual a 'Não informado'. O nome da coluna foi trocado para qtd_computador.

In [24]:
dicio_computador = {
    'A': 'Não.',
    'B': 'Sim, um.',
    'C': 'Sim, dois.',
    'D': 'Sim, três.',
    'E': 'Sim, quatro ou mais.',
    'F': 'Não informado'
}

df['Q024'] = (df['Q024'].fillna('F'))

df['Q024'] = df['Q024'].map(dicio_computador)

print("Coluna Q024")
print(df['Q024'].head(5))
print("Valores únicos da coluna 'Q024': \n", df['Q024'].unique())


Coluna Q024
0    Sim, um.
1        Não.
2    Sim, um.
3    Sim, um.
4    Sim, um.
Name: Q024, dtype: object
Valores únicos da coluna 'Q024': 
 ['Sim, um.' 'Não.' 'Sim, dois.' 'Sim, três.' 'Sim, quatro ou mais.'
 'Não informado']


In [25]:
df.rename(columns={'Q024': 'qtd_computador'}, inplace=True)
print("Coluna qtd_computador")
print(df['qtd_computador'].head(5))

Coluna qtd_computador
0    Sim, um.
1        Não.
2    Sim, um.
3    Sim, um.
4    Sim, um.
Name: qtd_computador, dtype: object
